# 🎯 Interview Coding Challenge 3: Streaming Data Quality Validation

**Difficulty:** Hard  
**Time:** 35 minutes  
**Skills:** Structured Streaming, data validation, error handling, monitoring

## Problem Statement

Implement real-time data quality validation for streaming data with comprehensive monitoring and alerting. Process streaming events and:

1. **Validate schema compliance** - Check required fields and data types
2. **Check data completeness** - Identify missing or null values
3. **Detect statistical anomalies** - Flag unusual patterns using rolling statistics
4. **Generate alerts** - Trigger notifications for quality issues
5. **Route data appropriately** - Send valid data to production, invalid to quarantine

## Input Schema
```python
event_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("value", DoubleType(), True),
    StructField("metadata", StringType(), True)
])
```

## Requirements
- Real-time processing with Structured Streaming
- Comprehensive quality checks with configurable thresholds
- Alert generation for different severity levels
- Data routing based on validation results
- Statistical anomaly detection using rolling windows

## Evaluation Criteria
- Correct streaming implementation
- Comprehensive validation logic
- Proper error handling and alerting
- Efficient data routing and monitoring

---

**Start coding your solution below!** 🚀

In [ ]:
# Initialize Spark Session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
from collections import defaultdict

spark = SparkSession.builder \
    .appName("Interview Challenge 3") \
    .getOrCreate()

print("Spark initialized successfully!")

In [ ]:
# Create sample streaming data for testing
streaming_data = [
    ("user1", "2023-01-01 10:00:00", "page_view", 100.0, "{}"),
    ("user2", "2023-01-01 10:01:00", "click", 50.0, "{}"),
    (None, "2023-01-01 10:02:00", "page_view", 75.0, "{}"),  # Missing user_id
    ("user3", None, "scroll", 25.0, "{}"),  # Missing timestamp
    ("user4", "2023-01-01 10:04:00", "purchase", 500.0, "{}"),  # Normal
    ("user5", "2023-01-01 10:05:00", "page_view", 9999.0, "{}"),  # Anomaly (very high value)
    ("user1", "2023-01-01 10:06:00", "logout", 0.0, "{}"),  # Normal
]

# Define expected schema
event_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("value", DoubleType(), True),
    StructField("metadata", StringType(), True)
])

# Create streaming DataFrame (simulate streaming with batch data)
events_df = spark.createDataFrame(streaming_data, 
    ["user_id", "timestamp", "event_type", "value", "metadata"])
events_df = events_df.withColumn("timestamp", to_timestamp("timestamp"))

print("Sample streaming data created:")
events_df.show()

## Your Solution

**Implement comprehensive streaming data quality validation:**

1. Create a `StreamingDataValidator` class with validation methods
2. Implement schema validation against expected schema
3. Check data completeness for required fields
4. Detect statistical anomalies using rolling windows
5. Generate alerts based on validation results
6. Route data to appropriate destinations (valid/invalid)

**Validation Rules:**
- Schema: Must match expected structure
- Completeness: < 5% nulls in required fields
- Anomalies: Values outside 3 standard deviations
- Critical alerts: Schema errors or high null rates

**Hints:**
- Use `foreachBatch` for micro-batch processing
- Implement statistical anomaly detection with window functions
- Create configurable alert thresholds
- Route data using conditional logic

In [ ]:
# YOUR SOLUTION HERE
# Implement the streaming data quality validation system

class StreamingDataValidator:
    def __init__(self, spark_session):
        self.spark = spark_session
        self.alert_thresholds = {
            "null_rate": 0.05,      # 5% null threshold
            "schema_error_rate": 0.001,  # 0.1% schema error threshold
            "anomaly_score": 3.0    # 3 standard deviations
        }
    
    def validate_schema(self, df, expected_schema):
        """Validate DataFrame schema against expected schema"""
        # YOUR CODE HERE - Compare actual vs expected schema
        # Return validation result with missing/extra/type mismatch fields
        pass
    
    def check_completeness(self, df, required_fields):
        """Check for null values in required fields"""
        # YOUR CODE HERE - Calculate null rates for each required field
        # Return list of completeness check results
        pass
    
    def detect_anomalies(self, df, numeric_field, window_size=10):
        """Detect statistical anomalies using rolling statistics"""
        # YOUR CODE HERE - Calculate rolling mean/std, z-scores
        # Flag values outside threshold standard deviations
        pass
    
    def generate_alerts(self, validation_results):
        """Generate alerts based on validation results"""
        # YOUR CODE HERE - Create alerts for different severity levels
        # Return list of alert dictionaries
        pass

def process_batch_with_validation(batch_df, batch_id):
    """Process a streaming batch with comprehensive validation"""
    
    validator = StreamingDataValidator(spark)
    
    # Define expected schema
    expected_schema = StructType([
        StructField("user_id", StringType(), True),
        StructField("timestamp", TimestampType(), True),
        StructField("event_type", StringType(), True),
        StructField("value", DoubleType(), True),
        StructField("metadata", StringType(), True)
    ])
    
    # Perform validations
    validation_results = {}
    
    # 1. Schema validation
    validation_results["schema"] = validator.validate_schema(batch_df, expected_schema)
    
    # 2. Data completeness
    validation_results["completeness"] = validator.check_completeness(
        batch_df, ["user_id", "timestamp", "event_type"]
    )
    
    # 3. Statistical anomalies (if value column exists)
    if "value" in batch_df.columns:
        validation_results["anomalies"] = validator.detect_anomalies(batch_df, "value")
    
    # Generate alerts
    alerts = validator.generate_alerts(validation_results)
    
    # Route data based on validation results
    critical_alerts = [a for a in alerts if a.get("severity") == "critical"]
    
    if critical_alerts:
        # Route to quarantine
        batch_df.withColumn("batch_id", lit(batch_id)) \
            .withColumn("quarantine_reason", lit(str(critical_alerts))) \
            .write.mode("append").parquet("/tmp/quarantine/")
        print(f"❌ Batch {batch_id} quarantined due to critical issues")
    else:
        # Route to production
        batch_df.withColumn("batch_id", lit(batch_id)) \
            .withColumn("quality_score", lit(len(alerts))) \
            .write.mode("append").parquet("/tmp/validated/")
        print(f"✅ Batch {batch_id} validated and stored")
    
    # Log alerts
    for alert in alerts:
        severity = alert.get("severity", "info").upper()
        print(f"[{severity}] {alert.get('message', 'Unknown alert')}")
    
    return batch_df

# Test your solution with sample data
try:
    result = process_batch_with_validation(events_df, batch_id=1)
    print("\n🎯 Validation processing completed!")
    print(f"Processed {events_df.count()} events")
    
except Exception as e:
    print(f"❌ Error in your solution: {e}")
    print("Keep working on your implementation!")

## Expected Solution Behavior

**Sample Output:**
```
[WARNING] High null rate in user_id: 14.29%
[WARNING] High null rate in timestamp: 14.29%
[INFO] Statistical anomalies detected: 1 events
✅ Batch 1 validated and stored
```

**Validation Results:**
- **Schema validation:** ✅ Passed (all fields present)
- **Data completeness:** ⚠️ Warnings for null user_id and timestamp
- **Statistical anomalies:** ℹ️ Info alert for high value (9999.0)
- **Data routing:** ✅ Valid data sent to production storage

---

**Outstanding work on this streaming validation challenge!** 🎉

**Key concepts tested:**
- Structured Streaming implementation
- Real-time data quality monitoring
- Statistical anomaly detection
- Alert generation and data routing
- Micro-batch processing patterns

**Production-ready features implemented:**
- Configurable validation thresholds
- Multi-severity alert system
- Data quarantine for invalid records
- Comprehensive monitoring and logging

**This challenge demonstrates advanced PySpark streaming capabilities!** 🚀